# Machine Learning Final Project Notebook with DenseNet 201
## *Petals to the Metal: Flower Classification on TPU*
### *By Xuanzhi Huang, Rahul Paul*

## Step 1: Some pre-setting

* ### Package preliminary

#### Import all the packages we need.

In [ ]:
# These are some basic packages
import random, re, math, os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt


# These are for data processing
import tensorflow_addons as tfa
from kaggle_datasets import KaggleDatasets


# These are for model training
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.applications import DenseNet201
import tensorflow.keras.backend as K


# These are performance metrics
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix


# These are for class weights
import datetime
import tqdm
import json
from collections import Counter
import gc

* ### Detect the hardware and tell the appropriate distribution strategy

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Make the system tune the number of threads for us
AUTO = tf.data.experimental.AUTOTUNE

* ### Configuration for image size, training epoch, batch size, and random seed

In [ ]:
IMAGE_SIZE = [512, 512]
EPOCHS = 16
SEED = 100
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

* ### Set the data access

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
# These are available image sizes in the data set
GCS_PATH_SELECT = { 
    192: GCS_DS_PATH + '/tfrecords-jpeg-192x192',
    224: GCS_DS_PATH + '/tfrecords-jpeg-224x224',
    331: GCS_DS_PATH + '/tfrecords-jpeg-331x331',
    512: GCS_DS_PATH + '/tfrecords-jpeg-512x512'
}
GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec')

* ### Add more mixed precision and/or XLA (refer to Chris Deotte's notebook)
[Rotation Augmentation GPU/TPU - [0.96+]](https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96)

In [ ]:
# Add more mixed precision and/or XLA to allow the TPU memory to handle larger batch sizes 
# and can speed up the training process
MIXED_PRECISION = False
XLA_ACCELERATE = False

if MIXED_PRECISION:
    if tpu: policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
    else: policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')

* ### Show all the classes we have

In [ ]:
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose'] 

## Step 2: Set some visualization functions

* ### Set training and validation curve function to show the changes in loss and accuracy

In [ ]:
def plot_train_valid_curves(training, validation, title, subplot):
    
    if subplot % 10 == 1:
        plt.subplots(figsize = (15,15), facecolor = '#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    ax.set_xlabel('epoch')
    ax.legend(['training', 'validation.'])

* ### Set a function to plot confusion matrix

In [ ]:
def display_confusion_matrix(cmat, score, precision, recall):
    
    plt.figure(figsize = (20,20))  # Specify the size of confusion matrix
    ax = plt.gca()
    ax.matshow(cmat, cmap = 'Reds')  # Draw a matrix
    ax.set_xticks(range(len(CLASSES)))  # Set the range of X coordinate according to #classes
    ax.set_xticklabels(CLASSES, fontdict={'fontsize': 7})  # Set the font size of X coordinate
    # Rotate labels on X coordinate to make them look better
    plt.setp(ax.get_xticklabels(), rotation = 45, ha = "left", rotation_mode = "anchor")
    ax.set_yticks(range(len(CLASSES)))  # Set the range of Y coordinate according to #classes
    ax.set_yticklabels(CLASSES, fontdict={'fontsize': 7})  # Set the font size of Y coordinate
    # Rotate labels on Y coordinate to make them look better
    plt.setp(ax.get_yticklabels(), rotation = 45, ha = "right", rotation_mode = "anchor")
    # Round F1 score, precision, and recall to the nearest fourth decimal place
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.4f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.4f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.4f} '.format(recall)
    # Add some comments about F1 score, precision, and recall on the plot
    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict = {'fontsize': 18, 'horizontalalignment': 'right', 'verticalalignment': 'top', 'color': 'Blue'})
    plt.show()

* ### Show the beautiful flowers (refer to Dimitre Oliveira)
[Flower with TPUs - Advanced augmentations](https://www.kaggle.com/dimitreoliveira/flower-with-tpus-advanced-augmentations)

In [ ]:
def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object:
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (for test data)
    return numpy_images, numpy_labels


def title_from_label_and_target_(label, correct_label):
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct


def display_one_flower(image, title, subplot, red = False, titlesize = 16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize = int(titlesize) if not red else int(titlesize / 1.2), color = 'red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2] + 1)


def display_batch_of_images(databatch, predictions = None):
    images, labels = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]
    rows = int(math.sqrt(len(images)))
    cols = len(images) // rows
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot = (rows,cols,1)
    if rows < cols:
        plt.figure(figsize = (FIGSIZE, FIGSIZE / cols*rows))
    else:
        plt.figure(figsize = (FIGSIZE / rows * cols,FIGSIZE))
    
    # Display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else CLASSES[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target_(predictions[i], label)
        dynamic_titlesize = FIGSIZE * SPACING / max(rows,cols) * 40 + 3
        subplot = display_one_flower(image, title, subplot, not correct, titlesize = dynamic_titlesize)
    
    # Layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace = 0, hspace = 0)
    else:
        plt.subplots_adjust(wspace = SPACING, hspace = SPACING)
    plt.show()
    

# Visualize model predictions (on training and validation sets)
# Images of flowers with labels telling whether prediction is true will be shown
def dataset_to_numpy_util(dataset, N):
    dataset = dataset.unbatch().batch(N)
    for images, labels in dataset:
        numpy_images = images.numpy()
        numpy_labels = labels.numpy()
        break;  
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    label = np.argmax(label, axis = -1)
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], str(correct), ', should be ' if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_flower_eval(image, title, subplot, red = False):
    plt.subplot(subplot)
    plt.axis('off')
    plt.imshow(image)
    plt.title(title, fontsize = 14, color = 'red' if red else 'black')
    return subplot + 1

def display_9_images_with_predictions(images, predictions, labels):
    subplot = 331
    plt.figure(figsize = (13,13))
    for i, image in enumerate(images):
        title, correct = title_from_label_and_target(predictions[i], labels[i])
        subplot = display_one_flower_eval(image, title, subplot, not correct)
        if i >= 8:
            break;
    plt.tight_layout()
    plt.subplots_adjust(wspace = 0.1, hspace = 0.1)
    plt.show()

## Step 3: Set functions to gain training set, validation set, and test set

* ### Decode images and convert pixels to floats between 0 and 1

In [ ]:
def decode_image(image_data):
    
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.cast(image, tf.float32) / 255.0
    # Reshape the images to fit the size required by TPU
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    
    return image

* ### Set a function to read labeled tfrec files (i.e. training & validation set)

In [ ]:
def read_labeled_tfrecord(example):
    
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "class": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    
    return image, label


# This is for data visualization
def read_labeled_id_tfrecord(example):
    
    LABELED_ID_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "class": tf.io.FixedLenFeature([], tf.int64),
        "id": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, LABELED_ID_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    idnum =  example['id']
    
    return image, label, idnum

* ### Set a function to read unlabeled tfrec files (i.e. test set)

In [ ]:
def read_unlabeled_tfrecord(example):
    
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "id": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    
    return image, idnum

* ### Load image data

In [ ]:
# For best performance, read from multiple tfrec files at once
# Disregard data's order, since data will be shuffled
def load_dataset(filenames, labeled = True, ordered = False):
    
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False  # Disable order to increase running speed
    # Automatically interleaves reading
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    # Use data in the shuffled order
    dataset = dataset.with_options(ignore_order)
    # Returns a dataset of (image, label) pairs if labeled = True (i.e. training & validation set)
    # or (image, id) pair if labeld = False (i.e. test set)
    dataset = dataset.map(read_labeled_id_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    
    return dataset

* ### Data augmentation

In [ ]:
# Randomly make some changes to the images and return the new images and labels
def data_augment(image, label):
        
    # Set seed for data augmentation
    seed = 100
    
    # Randomly resize and then crop images
    image = tf.image.resize(image, [720, 720])
    image = tf.image.random_crop(image, [512, 512, 3], seed = seed)

    # Randomly reset brightness of images
    image = tf.image.random_brightness(image, 0.6, seed = seed)
    
    # Randomly reset saturation of images
    image = tf.image.random_saturation(image, 3, 5, seed = seed)
        
    # Randomly reset contrast of images
    image = tf.image.random_contrast(image, 0.3, 0.5, seed = seed)

    # Randomly reset hue of images, but this will make the colors really weird, which we think will not happen
    # in common photography
    # image = tf.image.random_hue(image, 0.5, seed = seed)
    
    # Blur images
    image = tfa.image.mean_filter2d(image, filter_shape = 10)
    
    # Randomly flip images
    image = tf.image.random_flip_left_right(image, seed = seed)
    image = tf.image.random_flip_up_down(image, seed = seed)
    
    # Fail to rotate and transform images due to some bug in TensorFlow
    # angle = random.randint(0, 180)
    # image = tfa.image.rotate(image, tf.constant(np.pi * angle / 180))
    # image = tfa.image.transform(image, [1.0, 1.0, -250, 0.0, 1.0, 0.0, 0.0, 0.0])
    
    return image, label

* ### Gain training set

In [ ]:
def get_training_dataset():
   
    train = load_dataset(TRAINING_FILENAMES, labeled = True)
    train = train.map(lambda image, label, idnum: [image, label])
    train = train.repeat()
    train = train.shuffle(2048)
    train = train.batch(BATCH_SIZE)
    train = train.prefetch(AUTO)
    
    return train


# This function is for data visualization
def get_training_dataset_preview(ordered = True):
    
    train = load_dataset(TRAINING_FILENAMES, labeled = True, ordered = ordered)
    train = train.batch(BATCH_SIZE)
    train = train.cache()
    train = train.prefetch(AUTO)
    
    return train

* ### Gain validation set

In [ ]:
def get_validation_dataset(ordered = False):

    validation = load_dataset(VALIDATION_FILENAMES, labeled = True, ordered = ordered)
    validation = validation.map(lambda image, label, idnum: [image, label])
    validation = validation.batch(BATCH_SIZE)
    validation = validation.cache()
    # Prefetch next batch while training (autotune prefetch buffer size)
    validation = validation.prefetch(AUTO)
    
    return validation

* ### Gain test set

In [ ]:
def get_test_dataset(ordered = False):
    
    test = load_dataset(TEST_FILENAMES, labeled = False, ordered = ordered)
    test = test.batch(BATCH_SIZE)
    test = test.prefetch(AUTO)
    
    return test

* ### Count the number of images

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)  # Number of images in training set
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)  # Number of images in validation set
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)  # Number of images in test set
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE  # Steps of each epoch
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

* ### Show the beautiful flowers in training set before data augmentation

In [ ]:
train_dataset_aug = get_training_dataset()
display_batch_of_images(next(iter(train_dataset_aug.unbatch().batch(20))))
display_batch_of_images(next(iter(train_dataset_aug.unbatch().batch(20))))
display_batch_of_images(next(iter(train_dataset_aug.unbatch().batch(20))))

* ### Show the beautiful flowers in validation set before data augmentation

In [ ]:
validation_dataset_aug = get_validation_dataset()
display_batch_of_images(next(iter(validation_dataset_aug.unbatch().batch(20))))
display_batch_of_images(next(iter(validation_dataset_aug.unbatch().batch(20))))
display_batch_of_images(next(iter(validation_dataset_aug.unbatch().batch(20))))

* ### Show the beautiful flowers in test set before data augmentation

In [ ]:
test_dataset_aug = get_test_dataset()
display_batch_of_images(next(iter(test_dataset_aug.unbatch().batch(20))))
display_batch_of_images(next(iter(test_dataset_aug.unbatch().batch(20))))
display_batch_of_images(next(iter(test_dataset_aug.unbatch().batch(20))))

* ### Show example augmentation

In [ ]:
row = 3
col = 4
all_elements = get_training_dataset().unbatch()
one_element = tf.data.Dataset.from_tensors(next(iter(all_elements)))
# Map the images to the data augmentation function for image processing
augmented_element = one_element.repeat().map(data_augment).batch(row * col)

for (img, label) in augmented_element:
    plt.figure(figsize = (15, int(15 * row / col)))
    for j in range(row * col):
        plt.subplot(row, col, j + 1)
        plt.axis('off')
        plt.imshow(img[j, ])
    plt.show()
    break

## Step 4: Build the model and make prediction

* ### Customize learning rate scheduler and visualize it

In [ ]:
def lrfn(epoch):
    
    LR_START = 0.00001
    LR_MAX = 0.00005 * strategy.num_replicas_in_sync
    LR_MIN = 0.00001
    LR_RAMPUP_EPOCHS = 5
    LR_SUSTAIN_EPOCHS = 0
    LR_EXP_DECAY = .8
    
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY ** (epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

# Visualization changes in learning rate
rng = [i for i in range(25 if EPOCHS<25 else EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

* ### Build the model and load it into TPU

In [ ]:
with strategy.scope():
    # Create DenseNet201 model
    rnet = DenseNet201(
        input_shape = (512, 512, 3),
        weights = 'imagenet',  # Use the preset parameters of ImageNet
        include_top = False  # Drop the fully connected network on the top
    )
    
    rnet.trainable = True
    model = tf.keras.Sequential([
        rnet,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(len(CLASSES), activation = 'softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),  # Use Adam Algorithm for optimization
        # For multiclassification, we can use cross entropy or sparse cross entropy as our loss function 
        # These two cross entropy are the same in essence, but they are applied in different scenarios
        # If our target is one-hot encoded, it is better to use cross entropy
        # If our target is an integer, sparse cross entropy is a better choice, and this is our case
        loss = 'sparse_categorical_crossentropy', 
        metrics = ['sparse_categorical_accuracy']
    )

    model.summary()
    # Save the model
    model.save('ML_finalproject.h5')

* ### Calculate weight for each class (refer to Roman Weilguny)
[TPU flowers - First Love](https://www.kaggle.com/romanweilguny/tpu-flowers-first-love)

#### As the classes may not be uniformly distributed, add weights to classes

In [ ]:
gc.enable()

def get_training_dataset_raw():
    dataset = load_dataset(TRAINING_FILENAMES, labeled = True, ordered = False)
    return dataset

raw_training_dataset = get_training_dataset_raw()

label_counter = Counter()
for images, labels, id in raw_training_dataset:
    label_counter.update([labels.numpy()])

del raw_training_dataset    

TARGET_NUM_PER_CLASS = 122

def get_weight_for_class(class_id):
    counting = label_counter[class_id]
    weight = TARGET_NUM_PER_CLASS / counting
    return weight

weight_per_class = {class_id: get_weight_for_class(class_id) for class_id in range(104)}

* ### Train the model

In [ ]:
history = model.fit(
    get_training_dataset(),
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS,
    callbacks = [lr_callback],
    validation_data = get_validation_dataset(),
    class_weight = weight_per_class
)

* ### Show how loss and accuracy changes on training set

In [ ]:
plot_train_valid_curves(history.history['loss'], history.history['val_loss'], 'loss', 211)  # Loss curve
plot_train_valid_curves(history.history['sparse_categorical_accuracy'], 
                        history.history['val_sparse_categorical_accuracy'], 'accuracy', 212)  # Accuracy curve

* ### Check model's performance on validation set

#### Get the correct labels and predicted labels

In [ ]:
cmdataset = get_validation_dataset(ordered = True)
images_ds = cmdataset.map(lambda image, label: image)
labels_ds = cmdataset.map(lambda image, label: label).unbatch() 
cm_correct_labels = next(iter(labels_ds.batch(NUM_VALIDATION_IMAGES))).numpy()  # Get everything as one batch
cm_probabilities = model.predict(images_ds)  # The probability that each image is of each class
cm_predictions = np.argmax(cm_probabilities, axis = -1)  # The class of the largest probability is what we need
print("Correct labels: ", cm_correct_labels.shape, cm_correct_labels)
print("Predicted labels: ", cm_predictions.shape, cm_predictions)

#### Draw the confusion matrix, compute F1 score, precision, and recall

In [ ]:
cmat = confusion_matrix(cm_correct_labels, cm_predictions, labels = range(len(CLASSES)))
score = f1_score(cm_correct_labels, cm_predictions, labels = range(len(CLASSES)), average = 'macro')
precision = precision_score(cm_correct_labels, cm_predictions, labels = range(len(CLASSES)), average = 'macro')
recall = recall_score(cm_correct_labels, cm_predictions, labels = range(len(CLASSES)), average = 'macro')
cmat = (cmat.T / cmat.sum(axis = 1)).T
display_confusion_matrix(cmat, score, precision, recall)
print('f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(score, precision, recall))

* ### Make prediction

In [ ]:
test_ds = get_test_dataset(ordered = True)

test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds)  # Compute the probability that each image is of each class
predictions = np.argmax(probabilities, axis = -1)  # Use the one with largest probability as the predicted class
print(predictions)

# Generate submission file, remember to name it by "submission.csv"
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')
test = pd.DataFrame({"id": test_ids, "label": predictions})
print(test.head)
test.to_csv("submission.csv",index = False)